In [1]:
# install packages by pip or conda
# !pip install openai pykakasi python-Levenshtein unidecode

In [2]:
def get_normal_prompt(input_word):
    return f"""
    あなたはラッパーです。与えられた単語や短文に対してほとんどの母音が韻を踏んでいるような単語や短文を出力してください。ただし、出力は意味の通る単語か短文である必要があり、かつ入力と同じ単語を含んではいけません。
    input: {input_word}
    output:
    """

def get_better_prompt(input_word):
    return f'''
    あなたはラッパーです。与えられた単語や短文に対してほとんどの母音が韻を踏んでいるような単語や短文を出力してください。ただし、出力は意味の通る単語か短文である必要があり、かつ入力と同じ単語を含んではいけません。
    input: オートチューン
    output: 音の宇宙
    input: 皆のユートピア
    output: ミラノ風ドリア
    input: 幼稚園児の作文
    output: 広辞苑十冊分
    input: 押し倒して乗ります
    output: お慕いしております
    input: ノリとテンション
    output: 飲みニケーション
    input: 明日は危なくない
    output: 客はサクラくさい
    input: 小野小町
    output: 男泣き
    input: {input_word}
    output:
    '''

In [5]:
from openai import OpenAI
import unidecode
from pykakasi import kakasi
import re
from Levenshtein import distance
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()

# OpenAIのクライアントを設定
client = OpenAI()


# ローマ字変換の関数
def _to_romaji(text):
    kakasi_instance = kakasi()
    kakasi_instance.setMode("H", "a")  # Hiragana to ASCII
    kakasi_instance.setMode("K", "a")  # Katakana to ASCII
    kakasi_instance.setMode("J", "a")  # Japanese Kanji to ASCII
    converter = kakasi_instance.getConverter()
    return converter.do(text)


# 母音抽出の関数
def _extract_vowels(text):
    return "".join(re.findall(r"[aiueo]", text.lower()))


# GPT-4 APIを利用してプロンプトを処理する関数
def get_gpt_response(prompt: str) -> str:
    completion = client.chat.completions.create(
        model="gpt-4o", messages=[{"role": "user", "content": prompt}]
    )
    msg = completion.choices[0].message
    return msg.content


# レーベンシュタイン距離とスコアを計算する関数
def calculate_score(input_word: str, output_word: str) -> int:
    input_romaji = _to_romaji(input_word)
    output_romaji = _to_romaji(output_word)
    input_vowels = _extract_vowels(input_romaji)
    output_vowels = _extract_vowels(output_romaji)
    lev_distance = distance(input_vowels, output_vowels)
    score = max(0, 10 - lev_distance)

    return score

    # for debugging
    # return {
    #     "input_word": input_word,
    #     "output_word": output_word,
    #     "input_romaji": input_romaji,
    #     "output_romaji": output_romaji,
    #     "input_vowels": input_vowels,
    #     "output_vowels": output_vowels,
    #     "lev_distance": lev_distance,
    #     "score": score,
    # }


In [6]:
# テスト
test_input = "敵に塩"
test_output = "歴史にの"
print(calculate_score(test_input, test_output)) 
# >> 10

test_input = "皆のユートピア"
test_output = "ミラノ風ドリア" ## 「ふう」が「かぜ」になるせいで2点減点
print(calculate_score(test_input, test_output))

10
8


/var/folders/s9/cynxgzw13015wz0ft47fzbyc0000gn/T/ipykernel_51504/1190449088.py:18: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi_instance.setMode("H", "a")  # Hiragana to ASCII
/var/folders/s9/cynxgzw13015wz0ft47fzbyc0000gn/T/ipykernel_51504/1190449088.py:19: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi_instance.setMode("K", "a")  # Katakana to ASCII
/var/folders/s9/cynxgzw13015wz0ft47fzbyc0000gn/T/ipykernel_51504/1190449088.py:20: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi_instance.setMode("J", "a")  # Japanese Kanji to ASCII
/var/folders/s9/cynxgzw13015wz0ft47fzbyc0000gn/T/ipykernel_51504/1190449088.py:21: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version

In [9]:
input_words = [
    "ペンギンが踊る夢",
    "空飛ぶたこ焼き",
    "深夜のカラオケバトル",
    "猫が会議を仕切る" "恋のライバルはAI",
    "うさぎがDJする夜",
    "月面でサッカー観戦",
    "サボテンと朝ごはん",
    "ドラゴンと契約社員",
    "自転車で宇宙旅行"
    "砂漠でカフェ開業",
    "カエルがピアノ演奏",
    "ゴリラが書いた詩集",
    "雨の日の宇宙人",
    "ひよこが占い師",
    "トーストが恋をした",
    "海底でヨガレッスン",
    "鳥が語る哲学論",
    "ネコ耳社長の午後",
    "火星で花見大会",
]

total_score_for_normal_prompt = 0
total_score_for_better_prompt = 0
results_for_normal_prompt = []
results_for_better_prompt = []

for input in input_words:
    # for normal prompt
    normal_prompt: str = get_normal_prompt(input)
    normal_gpt_response: str = get_gpt_response(normal_prompt)
    results_for_normal_prompt.append(normal_gpt_response)
    score_for_normal: int = calculate_score(input, normal_gpt_response)
    total_score_for_normal_prompt += score_for_normal

    # for better prompt
    better_prompt: str = get_better_prompt(input)
    better_gpt_response: str = get_gpt_response(better_prompt)
    results_for_better_prompt.append(better_gpt_response)
    score_for_better: int = calculate_score(input, better_gpt_response)
    total_score_for_better_prompt += score_for_better

    print(f"input: {input}")
    print(f"normal response: {normal_gpt_response}")
    print(f"better response: {better_gpt_response}")
    print(f"score for normal prompt: {score_for_normal}")
    print(f"score for better prompt: {score_for_better}")
    print("")

/var/folders/s9/cynxgzw13015wz0ft47fzbyc0000gn/T/ipykernel_51504/1190449088.py:18: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi_instance.setMode("H", "a")  # Hiragana to ASCII
/var/folders/s9/cynxgzw13015wz0ft47fzbyc0000gn/T/ipykernel_51504/1190449088.py:19: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi_instance.setMode("K", "a")  # Katakana to ASCII
/var/folders/s9/cynxgzw13015wz0ft47fzbyc0000gn/T/ipykernel_51504/1190449088.py:20: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi_instance.setMode("J", "a")  # Japanese Kanji to ASCII
/var/folders/s9/cynxgzw13015wz0ft47fzbyc0000gn/T/ipykernel_51504/1190449088.py:21: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version

input: ペンギンが踊る夢
normal response: センチュリーの虹の夜
better response: 天気が戻る梅雨
score for normal prompt: 4
score for better prompt: 9

input: 空飛ぶたこ焼き
normal response: 高鳴る春の日
better response: ほら吹く神話劇
score for normal prompt: 5
score for better prompt: 5

input: 深夜のカラオケバトル
normal response: ミステリアスなソロライブ
better response: フィンランドのカラメルタルト
score for normal prompt: 4
score for better prompt: 5

input: 猫が会議を仕切る恋のライバルはAI
normal response: 人工天才が舞う愛の戦場で魅せる未来
better response: 明日、会いたいな、恋のカリスマは舞台
score for normal prompt: 0
score for better prompt: 1

input: うさぎがDJする夜
normal response: クラブで音楽鳴る光景
better response: サラミとチーズで乾杯
score for normal prompt: 4
score for better prompt: 2

input: 月面でサッカー観戦
normal response: 月面でサッカー観戦  
こんな奇跡に乾杯の宣言
better response: 律令で合唱団全員
score for normal prompt: 0
score for better prompt: 4

input: サボテンと朝ごはん
normal response: カーテンの長さ指南
better response: 箱根でパラソルさん
score for normal prompt: 7
score for better prompt: 8

input: ドラゴンと契約社員
normal response: ドラマで上役も驚愕达人
better response: 誰かの声援
score for

In [10]:
print(f"total score for normal prompt: {total_score_for_normal_prompt}")
print(f"total score for better prompt: {total_score_for_better_prompt}")

total score for normal prompt: 67
total score for better prompt: 94
